In [ ]:
# Auto-reload modules before executing code
%load_ext autoreload
%autoreload 2

import requests
import json
from pprint import pprint

## Configuration

Set the base URL for the API (assumes server is running locally).

In [ ]:
BASE_URL = "http://localhost:8000"
API_URL = f"{BASE_URL}/api/v1/risks"

print(f"API Base URL: {API_URL}")
print("\nMake sure the server is running with:")
print("  uv run python -m reit_risk_summarizer.main")

## Test 1: Health Check

Verify the API is running.

In [ ]:
response = requests.get(f"{BASE_URL}/health")
print(f"Status Code: {response.status_code}")
pprint(response.json())

## Test 2: Get Risks for AMT (First Call)

Process American Tower - should fetch, extract, and summarize.

In [ ]:
ticker = "AMT"
response = requests.get(f"{API_URL}/{ticker}")

print(f"Status Code: {response.status_code}")
print(f"\nResponse:")
if response.status_code == 200:
    data = response.json()
    print(f"Ticker: {data['ticker']}")
    print(f"Company: {data['company_name']}")
    print(f"Model: {data['model']}")
    print(f"Cached: {data['cached']}")
    print(f"\nTop 5 Risks:")
    for i, risk in enumerate(data['risks'], 1):
        print(f"{i}. {risk[:100]}...")  # First 100 chars
else:
    print(response.text)

## Test 3: Get Risks for AMT Again (Cache Hit)

Should return cached results much faster.

In [ ]:
import time

start = time.time()
response = requests.get(f"{API_URL}/{ticker}")
elapsed = time.time() - start

print(f"Status Code: {response.status_code}")
print(f"Response Time: {elapsed:.2f}s")

if response.status_code == 200:
    data = response.json()
    print(f"Cached: {data['cached']}")
    print(f"\nTop 5 Risks:")
    for i, risk in enumerate(data['risks'], 1):
        print(f"{i}. {risk[:100]}...")  # First 100 chars

## Test 4: Force Refresh

Bypass cache using `force_refresh=true` query parameter.

In [ ]:
response = requests.get(f"{API_URL}/{ticker}", params={"force_refresh": True})

print(f"Status Code: {response.status_code}")
if response.status_code == 200:
    data = response.json()
    print(f"Cached: {data['cached']}")
    print(f"Model: {data['model']}")

## Test 5: Different Ticker (PLD)

Process Prologis.

In [ ]:
ticker = "PLD"
response = requests.get(f"{API_URL}/{ticker}")

print(f"Status Code: {response.status_code}")
if response.status_code == 200:
    data = response.json()
    print(f"\nTicker: {data['ticker']}")
    print(f"Company: {data['company_name']}")
    print(f"Cached: {data['cached']}")
    print(f"\nTop 5 Risks:")
    for i, risk in enumerate(data['risks'], 1):
        print(f"{i}. {risk[:100]}...")  # First 100 chars

## Test 6: Invalid Ticker (Error Handling)

Test 404 error handling.

In [ ]:
response = requests.get(f"{API_URL}/INVALID_XYZ_123")

print(f"Status Code: {response.status_code}")
print(f"\nError Response:")
pprint(response.json())

## Test 7: Clear Cache for Specific Ticker

Delete cache using DELETE endpoint.

In [ ]:
ticker = "AMT"
response = requests.delete(f"{API_URL}/cache/{ticker}")

print(f"Status Code: {response.status_code}")
print(f"Expected: 204 No Content")

# Verify cache was cleared by checking next request
print(f"\nVerifying cache was cleared...")
response = requests.get(f"{API_URL}/{ticker}")
if response.status_code == 200:
    data = response.json()
    print(f"Cached: {data['cached']} (should be False)")

## Test 8: Clear All Cache

Clear entire cache.

In [ ]:
response = requests.delete(f"{API_URL}/cache")

print(f"Status Code: {response.status_code}")
print(f"Expected: 204 No Content")
print("\nAll cache cleared successfully")

## Test 9: API Documentation

Check OpenAPI schema.

In [ ]:
response = requests.get(f"{BASE_URL}/openapi.json")

print(f"Status Code: {response.status_code}")
if response.status_code == 200:
    schema = response.json()
    print(f"\nAPI Title: {schema['info']['title']}")
    print(f"Version: {schema['info']['version']}")
    print(f"\nAvailable Endpoints:")
    for path, methods in schema['paths'].items():
        for method in methods.keys():
            print(f"  {method.upper()} {path}")

## Test 10: Batch Processing

Process multiple tickers in sequence.

In [ ]:
tickers = ["AMT", "PLD", "EQIX"]
results = {}

for ticker in tickers:
    print(f"\nProcessing {ticker}...")
    start = time.time()
    response = requests.get(f"{API_URL}/{ticker}")
    elapsed = time.time() - start
    
    if response.status_code == 200:
        data = response.json()
        results[ticker] = {
            "company": data['company_name'],
            "cached": data['cached'],
            "time": elapsed,
            "status": "✓ Success"
        }
        print(f"  Company: {data['company_name']}")
        print(f"  Cached: {data['cached']}")
        print(f"  Time: {elapsed:.2f}s")
    else:
        results[ticker] = {"status": f"✗ Error {response.status_code}"}
        print(f"  Error: {response.status_code}")

print("\n" + "="*60)
print("Summary:")
for ticker, result in results.items():
    print(f"{ticker}: {result['status']}")